In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [4]:
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [7]:
test.isnull().sum().sum()

0

In [10]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [11]:
x_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"]

In [12]:
x_train = x_train.values.reshape(-1,28,28,1)
x_test = test.values.reshape(-1,28,28,1)

In [13]:
x_train=x_train/x_train.max()
x_test=x_test/x_test.max()

In [14]:
from keras.utils.np_utils import to_categorical

In [15]:
y_cat_train=to_categorical(y_train,10)

In [16]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten

In [17]:
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=(4,4),input_shape=(28,28,1),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(4,4),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(10,activation="softmax"))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 64)        1088      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          65600     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 128)               1

In [19]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.fit(x_train,y_cat_train,epochs=50,validation_split=0.2)

Epoch 1/50
1050/1050 [==============================] - 38s 35ms/step - loss: 0.1703 - accuracy: 0.9475 - val_loss: 0.0665 - val_accuracy: 0.9779
Epoch 2/50
1050/1050 [==============================] - 37s 35ms/step - loss: 0.0515 - accuracy: 0.9836 - val_loss: 0.0537 - val_accuracy: 0.9831
Epoch 3/50
1050/1050 [==============================] - 36s 34ms/step - loss: 0.0334 - accuracy: 0.9896 - val_loss: 0.0367 - val_accuracy: 0.9888
Epoch 4/50
1050/1050 [==============================] - 35s 33ms/step - loss: 0.0246 - accuracy: 0.9919 - val_loss: 0.0374 - val_accuracy: 0.9883
Epoch 5/50
1050/1050 [==============================] - 35s 33ms/step - loss: 0.0191 - accuracy: 0.9940 - val_loss: 0.0367 - val_accuracy: 0.9890
Epoch 6/50
1050/1050 [==============================] - 35s 33ms/step - loss: 0.0138 - accuracy: 0.9956 - val_loss: 0.0488 - val_accuracy: 0.9873
Epoch 7/50
1050/1050 [==============================] - 35s 33ms/step - loss: 0.0119 - accuracy: 0.9961 - val_loss: 0.0428 -

In [ ]:
preds=model.predict(x_test)

In [ ]:
preds=np.argmax(preds,axis=1)
preds

In [ ]:
preds=pd.Series(data=preds)

In [ ]:
Label=preds
ImageId=np.arange(1,28001,1)
ImageId=pd.Series(data=ImageId)

In [ ]:
submit=pd.concat([ImageId,Label],axis=1)
submit.rename(columns={0: "ImageId"},inplace=True)
submit.rename(columns={1: "Label"},inplace=True)
submit

In [ ]:
submit.to_csv('submission.csv',index=False)